In [ ]:
import speech_recognition as sr
import re
import pyttsx3
import speech_recognition as sr
import geocoder
import requests
import json
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


def trip():
    engine = pyttsx3.init('sapi5')
    engine.say("Where do you want to go")
    engine.runAndWait()
    rate = 150
    engine.setProperty('rate', rate)
    volume = 0.9
    engine.setProperty('volume', volume)
    voices = engine.getProperty('voices')
    engine.setProperty('voice', voices[0].id)
    r = sr.Recognizer()
    mic = sr.Microphone()

    with mic as source:
        r.adjust_for_ambient_noise(source)
        print("Please give input as from your current location to destination ")
        print("Please state your current location and destination:")
        audio = r.listen(source)

    try:
        user_input = r.recognize_google(audio)
        print("User input:", user_input)
    except sr.UnknownValueError:
        print("Unable to recognize speech")
        return
    except sr.RequestError as e:
        print("Error:", e)
        return

    location_regex = r'from\s([\w\s]+)\sto'
    destination_regex = r'to\s([\w\s]+)'

    location_match = re.search(location_regex, user_input, re.IGNORECASE)
    destination_match = re.search(destination_regex, user_input, re.IGNORECASE)

    if location_match and destination_match:
        current_location = location_match.group(1).strip()
        destination = destination_match.group(1).strip()
        print("Current location:", current_location)
        print("Destination:", destination)
    else:
        print("Unable to extract location and destination from user input")
        return

    driver = webdriver.Chrome()

    url = f"https://www.google.com/maps/dir/?api=1&origin={current_location}&destination={destination}&travelmode=transit"

    driver.get(url)

    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "section-directions-trip-description")))

    transit_directions = driver.find_elements_by_class_name("section-directions-trip-description")

    for transit in transit_directions:
        print(transit.text)

    driver.quit()


In [ ]:
trip()